In [60]:
import json
import random
from collections import defaultdict
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold

In [61]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [62]:
# Copy-pasted from the game.
# https://heroes.cdnvideo.ru/vk/v0326/locale/ru.json.gz
lib = {
  "LIB_HERO_NAME_0": "Карен",
  "LIB_HERO_NAME_1": "Аврора",
  "LIB_HERO_NAME_10": "Безликий",
  "LIB_HERO_NAME_11": "Чабба",
  "LIB_HERO_NAME_12": "Арахна",
  "LIB_HERO_NAME_13": "Орион",
  "LIB_HERO_NAME_14": "Фокс",
  "LIB_HERO_NAME_15": "Джинджер",
  "LIB_HERO_NAME_16": "Данте",
  "LIB_HERO_NAME_17": "Моджо",
  "LIB_HERO_NAME_18": "Судья",
  "LIB_HERO_NAME_19": "Темная Звезда",
  "LIB_HERO_NAME_2": "Галахад",
  "LIB_HERO_NAME_20": "Артемис",
  "LIB_HERO_NAME_21": "Маркус",
  "LIB_HERO_NAME_22": "Пеппи",
  "LIB_HERO_NAME_23": "Лиэн",
  "LIB_HERO_NAME_24": "Тесак",
  "LIB_HERO_NAME_25": "Исмаил",
  "LIB_HERO_NAME_26": "Лилит",
  "LIB_HERO_NAME_27": "Лютер",
  "LIB_HERO_NAME_28": "Цин Мао",
  "LIB_HERO_NAME_29": "Дориан",
  "LIB_HERO_NAME_3": "Кира",
  "LIB_HERO_NAME_30": "Корнелиус",
  "LIB_HERO_NAME_31": "Джет",
  "LIB_HERO_NAME_32": "Гелиос",
  "LIB_HERO_NAME_33": "Ларс",
  "LIB_HERO_NAME_34": "Криста",
  "LIB_HERO_NAME_35": "Йорген",
  "LIB_HERO_NAME_36": "Майя",
  "LIB_HERO_NAME_37": "Джу",
  "LIB_HERO_NAME_38": "Эльмир",
  "LIB_HERO_NAME_39": "Зири",
  "LIB_HERO_NAME_4": "Астарот",
  "LIB_HERO_NAME_40": "Небула",
  "LIB_HERO_NAME_41": "К'арх",
  "LIB_HERO_NAME_5": "Кай",
  "LIB_HERO_NAME_6": "Фобос",
  "LIB_HERO_NAME_7": "Тея",
  "LIB_HERO_NAME_8": "Сорвиголова",
  "LIB_HERO_NAME_9": "Хайди",
}

In [63]:
def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    for battle_key, sign in (('player', +1), ('enemies', -1)):
        for hero in battle[battle_key]:
            for hero_key in ('Level', 'Color', 'Star'):
                result[f'''{lib[f"LIB_HERO_NAME_{hero['id']}"]} {hero_key}'''] += sign * hero[hero_key.lower()]
        
    return {'Win': battle['win'], **result}

In [64]:
def invert_column(series: pandas.Series):
    """
    Inverts the column to make an "opposite" battle.
    """
    return series == False if series.name == 'Win' else -series

random.seed(42)
battles = pandas.DataFrame([parse_battle(line) for line in open('battles.jsonl')]).fillna(value=0)
battles = pandas.concat((battles, battles.apply(invert_column)))
battles.head()

,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
0,False,0.0,0.0,0.0,4.0,35.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.0,0.0,0.0,-2.0,-10.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,0.0,0.0,0.0,4.0,36.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-45.0,-2.0
3,False,0.0,0.0,0.0,-1.0,-10.0,-1.0,-4.0,-46.0,-3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.0,0.0,0.0,-1.0,-4.0,-2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
battles.describe()

,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,Астарот Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
count,278.000000,278.000000,278.000000,278.000000,278.00000,278.000000,278.000000,278.000000,278.000000,278.000000,...,278.000000,278.000000,278.000000,278.00000,278.000000,278.000000,278.000000,278.000000,278.00000,278.000000
mean,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
std,2.471498,19.640634,0.870702,3.549546,28.27342,1.718449,3.549546,29.945317,1.285858,5.140622,...,0.268705,1.084848,8.938216,0.30637,1.411659,11.228188,0.624413,0.942383,7.90284,0.398553
min,-8.000000,-61.000000,-3.000000,-8.000000,-70.00000,-6.000000,-8.000000,-69.000000,-3.000000,-8.000000,...,-2.000000,-7.000000,-60.000000,-2.00000,-8.000000,-61.000000,-3.000000,-7.000000,-60.00000,-3.000000
25%,-0.000000,-0.000000,-0.000000,-1.000000,-4.00000,-0.000000,-0.000000,-0.000000,-0.000000,-6.000000,...,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,4.00000,0.000000,0.000000,0.000000,0.000000,6.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,8.000000,61.000000,3.000000,8.000000,70.00000,6.000000,8.000000,69.000000,3.000000,8.000000,...,2.000000,7.000000,60.000000,2.00000,8.000000,61.000000,3.000000,7.000000,60.00000,3.000000


In [66]:
x = battles.drop(['Win'], axis=1)
y = battles['Win']

In [67]:
classifier = LogisticRegression(max_iter=1000, fit_intercept=False)

numpy.random.seed(42)
grid_search = GridSearchCV(
    classifier,
    {'C': numpy.logspace(-6, 2, num=1000)},
    cv=cv,
    scoring='accuracy',
).fit(x, y)

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {grid_search.best_estimator_.classes_}')

scores = cross_val_score(grid_search.best_estimator_, x, y, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

Best score: 0.9784172661870504
Best params: {'C': 38.333951017665974}
Classes: [False  True]
CV score: 0.9780219780219781 (std: 0.033601944728397025)


In [68]:
pandas.DataFrame({'Feature': x.columns, 'Importance': grid_search.best_estimator_.coef_[0]}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=20)

,Importance
Feature,
Астарот Star,9.494312
Судья Color,5.981885
Фобос Color,5.532145
Джинджер Color,5.075376
Исмаил Star,4.912910
Арахна Color,4.910078
Сорвиголова Color,4.267968
Галахад Star,4.000067
Сорвиголова Star,3.490914


In [69]:
result = pandas.concat((
    pandas.Series(grid_search.best_estimator_.predict(x), index=battles.index, name='Predicted'),
    pandas.Series(grid_search.best_estimator_.predict_proba(x)[:, 1], index=battles.index, name='Probability'),
    battles,
), axis=1)
result['Probability'] = result['Probability'].apply('{:.2f}'.format)
result.head()

,Predicted,Probability,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
0,False,0.00,False,0.0,0.0,0.0,4.0,35.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.00,False,0.0,0.0,0.0,-2.0,-10.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,0.00,False,0.0,0.0,0.0,4.0,36.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-45.0,-2.0
3,False,0.02,False,0.0,0.0,0.0,-1.0,-10.0,-1.0,-4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.00,False,0.0,0.0,0.0,-1.0,-4.0,-2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
# Experimental NN model.
# I want it to catch an interaction between different heroes (e.g. Йорген is good in conjunction with Исмаил).

def create_model() -> keras.Model:
    model = Sequential()
    model.add(Dense(1000, input_dim=x.columns.size, activation='sigmoid'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(1, input_dim=x.columns.size, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

keras_classifier = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)

numpy.random.seed(42)
scores = cross_val_score(keras_classifier, x, y, cv=cv)

print(f'Score: {scores.mean()} (std: {scores.std()})')

Score: 0.9637362599372864 (std: 0.03259405159943645)
